# Data Engineering Capstone Project - Metro_Madrid

## Project Summary
This notebook gets output data from [metro-big-data-unir](https://github.com/juananthony/metro-big-data-unir) project and create a database to analysis.

[*Metro de Madrid*](https://www.metromadrid.es/) is the name of the tube/subway service that operates in Madrid, Spain. This service has 302 stations on 13 lines plus a light rail system called *Metro Ligero*. This service is used, on average in 2020, more than 30 million times each month.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('udacity-capstone').getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'CET')

## Step 1: Scope the Project and Gather Data

### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
DATA_PATH = './data'
lines_file = 'lines.csv'
stations_file = 'stations.csv'
mentions_file = 'mentions_20210210.csv'

In [4]:
lines = spark.read.csv(os.path.join(DATA_PATH, lines_file), header=True)
lines.count()

16

In [5]:
stations = spark.read.csv(os.path.join(DATA_PATH, stations_file), header=True)
stations.count()

332

In [6]:
mentions = spark.read.csv(os.path.join(DATA_PATH, mentions_file), header=True, multiLine=True, escape='"')
mentions.show()

+--------------------+--------------------+--------------------+-------------------+---------------------+------------------+--------------------+----------------+--------------------+----------------------+----------------------------+------------------------+---------------------------------+
|                 _id|          created_at|                text|                 id|in_reply_to_status_id|           user.id|           user.name|user.screen_name|    user.description|user.profile_image_url|user.profile_image_url_https|extended_tweet.full_text|classification.naive_bayes.result|
+--------------------+--------------------+--------------------+-------------------+---------------------+------------------+--------------------+----------------+--------------------+----------------------+----------------------------+------------------------+---------------------------------+
|ObjectId(5be9f6d0...|Mon Nov 12 21:55:...|@metro_madrid Qué...|1062101577610215424|  1060150880522121216|702578

In [7]:
mentions.printSchema()

root
 |-- _id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- user.id: string (nullable = true)
 |-- user.name: string (nullable = true)
 |-- user.screen_name: string (nullable = true)
 |-- user.description: string (nullable = true)
 |-- user.profile_image_url: string (nullable = true)
 |-- user.profile_image_url_https: string (nullable = true)
 |-- extended_tweet.full_text: string (nullable = true)
 |-- classification.naive_bayes.result: string (nullable = true)



In [8]:
mentions.groupBy('`classification.naive_bayes.result`').count().show()

+---------------------------------+------+
|classification.naive_bayes.result| count|
+---------------------------------+------+
|                             null| 85465|
|                          nothing|698591|
|                        complaint|111612|
|                            issue| 59499|
+---------------------------------+------+



## Step 2: Explore and Assess the Data
### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

### Cleaning Steps
Document steps necessary to clean the data

In [9]:
from pyspark.sql.functions import col, to_timestamp
classes = ['complaint', 'issue']
mentions = mentions.filter(col('`classification.naive_bayes.result`').isin(classes))

In [10]:
mentions = mentions.withColumn('dt', to_timestamp(mentions.created_at, 'E MMM d HH:m:s Z y').alias('dt'))

In [11]:
from pyspark.sql.functions import when
mentions = mentions.withColumn('full_text', 
                               when(~col('`extended_tweet.full_text`').isNull(), col('`extended_tweet.full_text`'))
                               .otherwise(col('text'))).drop('text','`extended_tweet.full_text`')

## Step 3: Define the Data Model
### 3.1 Conceptual Data Model
The data we want to store is all messages that inform about any issue or complaint in a line or a station even if one message inform about an issue that affect two different lines. That the reason why the fact table is the inform fact, that can be a complaint or an issue. One tweet can inform about an issue that affect two lines (i.e.: a closed station and all lines that stops there). In other words, one tweet generates one or many "inform fact" records.

![fact-dimension diagram](./img/class_diagram.png "Fact-Dimension Diagram")

### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
 
#### Line Dimension

## Step 4: Run ETL to model the data

### 4.1 Create the data model
Build the data pipelines to create the data model.

#### Line Dimension

In [12]:
from pyspark.sql.functions import monotonically_increasing_id
lines_dim = lines.withColumn('line_id', monotonically_increasing_id()).withColumnRenamed('line','line_name')
lines_dim.show(5)

+---------+-------+
|line_name|line_id|
+---------+-------+
|       L9|      0|
|      ML3|      1|
|       L8|      2|
|      ML1|      3|
|      ML2|      4|
+---------+-------+
only showing top 5 rows



#### Station Dimension

In [13]:
stations_dim = stations.join(lines_dim, lines_dim.line_name == stations.line, 'inner').select(monotonically_increasing_id().alias('station_id'), stations.station.alias('station_name'), lines_dim.line_id)
stations_dim.show(5)

+----------+----------------+-------+
|station_id|    station_name|line_id|
+----------+----------------+-------+
|         0|   Paco de Lucía|      0|
|         1|      Mirasierra|      0|
|         2|    Herrera Oria|      0|
|         3|Barrio del Pilar|      0|
|         4|        Ventilla|      0|
+----------+----------------+-------+
only showing top 5 rows



#### Class Diagram

In [28]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number

class_df = mentions.select(col('`classification.naive_bayes.result`').alias('class_name')).distinct()
w = Window.orderBy('class_name')

class_dim = class_df.withColumn('class_id', row_number().over(w))

#### Mapping date dimensions

In [24]:
from pyspark.sql.functions import year, month, quarter, dayofweek, dayofmonth, hour, minute

date_dim = mentions.select(
                        year(col('dt')).alias('year'),
                        quarter(col('dt')).alias('quarter'),
                        month(col('dt')).alias('month'),
                        dayofmonth(col('dt')).alias('day'),
                        dayofweek(col('dt')).alias('weekday'),
                        hour(col('dt')).alias('hour'),
                        minute(col('dt')).alias('minute')) \
                .dropDuplicates() \
                .withColumn('date_id', monotonically_increasing_id())

date_dim.show(10, False)

+----+-------+-----+---+-------+----+------+-------+
|year|quarter|month|day|weekday|hour|minute|date_id|
+----+-------+-----+---+-------+----+------+-------+
|2018|4      |11   |17 |7      |19  |42    |0      |
|2018|4      |11   |20 |3      |11  |56    |1      |
|2018|4      |11   |26 |2      |12  |22    |2      |
|2018|4      |11   |30 |6      |0   |24    |3      |
|2018|4      |11   |30 |6      |14  |35    |4      |
|2018|4      |11   |30 |6      |19  |47    |5      |
|2018|4      |11   |30 |6      |20  |44    |6      |
|2018|4      |12   |1  |7      |9   |3     |7      |
|2018|4      |12   |6  |5      |9   |50    |8      |
|2018|4      |12   |6  |5      |10  |56    |9      |
+----+-------+-----+---+-------+----+------+-------+
only showing top 10 rows



#### User Dimension

In [23]:
user_dim = mentions.select(
            col('`user.id`').alias('user_id'),
            col('`user.name`').alias('user_name'),
            col('`user.screen_name`').alias('screen_name'),
            col('`user.description`').alias('description'),
            col('`user.profile_image_url`').alias('profile_image_url'),
            col('`user.profile_image_url_https`').alias('profile_image_url_https')
)

#### Tweet Dimension

In [26]:
from pyspark.sql.functions import substring

tweet_dim = mentions.join(date_dim,
              (year(mentions.dt) == date_dim.year) &
              (month(mentions.dt) == date_dim.month) &
              (dayofmonth(mentions.dt) == date_dim.day) &
              (hour(mentions.dt) == date_dim.hour) &
              (minute(mentions.dt) == date_dim.minute),
              'inner'
             ) \
        .select(col('id').alias('tweet_id'),
                'date_id',
                col('`user.id`').alias('user_id'),
                col('full_text').alias('text'),
                col('in_reply_to_status_id').alias('reply_tweet_id')
               )

#### Fact table

In [63]:
import re
def generate_line_tags(text):
    regex_dict = {
        "L1": "l(i|í)neas? ?(^|\D)1(\D|$)| L ?(^|\D)1(\D|$)|Pinar ?de ?Chamart(i|í)n|Bamb(u|ú)|Chamart(i|í)n|Plaza ?de ?Castilla|Valdeacederas|Tetu(a|á)n|Estrecho|Alvarado|Cuatro ?Caminos|R(i|í)os ?Rosas|Iglesia	|Bilbao|Tribunal|Gran ?V(i|í)a|Sol|Tirso ?de ?Molina|Ant(o|ó)n ?Mart(i|í)n|Estaci(o|ó)n ?del ?Arte|Atocha ?Renfe|Men(e|é)ndez ?Pelayo|Pac(i|í)fico|Puente ?de ?Vallecas|Nueva ?Numancia|Portazgo|Buenos ?Aires|Alto ?del ?Arenal|Miguel ?Hern(a|á)ndez|Sierra ?de ?Guadalupe|Villa ?de ?Vallecas|Congosto|La ?Gavia|Las ?Suertes|Valdecarros",
        "L2": "l(i|í)neas? ?(^|\D)2(\D|$)| L ?(^|\D)2(\D|$)|Las ?Rosas|Avenida ?de ?Guadalajara|Alsacia|La ?Almudena|La ?Elipa|Ventas|Manuel ?Becerra|Goya|Pr(i|í)ncipe ?de ?Vergara|Retiro|Banco ?de ?España|Sevilla|Sol|Ópera|Santo ?Domingo|Noviciado|San ?Bernardo|Quevedo|Canal|Cuatro ?Caminos",
        "L3": "l(i|í)neas? ?(^|\D)3(\D|$)| L ?(^|\D)3(\D|$)|Villaverde|San ?Crist(o|ó)bal|Villaverde ?Bajo-Cruce|Ciudad ?de ?los ?Ángeles|San ?Ferm(i|í)n-Orcasur|Hospital ?12 ?de ?Octubre|Almendrales|Legazpi|Delicias|Palos ?de ?la ?Frontera|Embajadores|Lavapi(e|é)s|Sol|Callao|Plaza ?de ?España|Ventura ?Rodr(i|í)guez|Argüelles|Moncloa",
        "L4": "l(i|í)neas? ?(^|\D)4(\D|$)| L ?(^|\D)4(\D|$)|Argüelles|San ?Bernardo|Bilbao|Alonso ?Mart(i|í)nez|Col(o|ó)n|Serrano|Vel(a|á)zquez|Goya|Lista|Diego ?de ?Le(o|ó)n|Avenida ?de ?Am(e|é)rica|Prosperidad|Alfonso ?XIII|Avenida ?de ?la ?Paz|Arturo ?Soria|Esperanza|Canillas|Mar ?de ?Cristal|San ?Lorenzo		|Parque ?de ?Santa ?Mar(i|í)a|Hortaleza|Manoteras|Pinar ?de ?Chamart(i|í)n",
        "L5": "l(i|í)neas? ?(^|\D)5(\D|$)| L ?(^|\D)5(\D|$)|Alameda ?de ?Osuna|El ?Capricho|Canillejas|Torre ?Arias|Suanzes|Ciudad ?Lineal|Pueblo ?Nuevo|Quintana|El ?Carmen|Ventas|Diego ?de ?Le(o|ó)n|N(u|ú)ñez ?de ?Balboa|Rub(e|é)n ?Dar(i|í)o|Alonso ?Mart(i|í)nez|Chueca|Gran ?V(i|í)a|Callao|Ópera|La ?Latina|Puerta ?de ?Toledo|Acacias|Pir(a|á)mides|Marqu(e|é)s ?de ?Vadillo|Urgel|Oporto|Vista ?Alegre|Carabanchel|Eugenia ?de ?Montijo|Aluche|Empalme|Campamento|Casa ?de ?Campo",
        "L6": "l(i|í)neas? ?(^|\D)6(\D|$)| L ?(^|\D)6(\D|$)|Laguna|Lucero|Carpetana|Alto ?de ?Extremadura|Oporto|Puerta ?del ?Ángel|Opañel|Plaza ?El(i|í)ptica|Pr(i|í)ncipe ?P(i|í)o|Usera|Argüelles|Moncloa|Legazpi|Ciudad ?Universitaria|Arganzuela-Planetario|Vicente ?Aleixandre|M(e|é)ndez ?Álvaro|Guzm(a|á)n ?el ?Bueno|Pac(i|í)fico|Cuatro ?Caminos|Conde ?de ?Casal|Nuevos ?Ministerios|Sainz ?de ?Baranda|Rep(u|ú)blica ?Argentina|O'Donnell|Avenida ?de ?Am(e|é)rica|Manuel ?Becerra|Diego ?de ?Le(o|ó)n",
        "L7": "l(i|í)neas? ?(^|\D)7(\D|$)| L ?(^|\D)7(\D|$)|Hospital ?del ?Henares|Henares|Jarama|San ?Fernando|La ?Rambla|Coslada ?Central|Barrio ?del ?Puerto|Estadio ?Metropolitano|Las ?Musas|San ?Blas|Simancas|Garc(i|í)a ?Noblejas|Ascao|Pueblo ?Nuevo|Barrio ?de ?la ?Concepci(o|ó)n|Parque ?de ?las ?Avenidas|Cartagena|Avenida ?de ?Am(e|é)rica|Gregorio ?Marañ(o|ó)n|Alonso ?Cano|Canal|Islas ?Filipinas|Guzm(a|á)n ?el ?Bueno|Francos ?Rodr(i|í)guez|Valdezarza|Antonio ?Machado|Peñagrande|Avenida ?de ?la ?Ilustraci(o|ó)n|Lacoma|Arroyofresno|Pitis",
        "L8": "l(i|í)neas? ?(^|\D)8(\D|$)| L ?(^|\D)8(\D|$)|Nuevos ?Ministerios|Colombia|Pinar ?del ?Rey|Mar ?de ?Cristal|Feria ?de ?Madrid|Aeropuerto ?T1-T2-T3|Barajas|Aeropuerto ?T4",
        "L9": "l(i|í)neas? ?(^|\D)9(\D|$)| L ?(^|\D)9(\D|$)|Paco ?de ?Luc(i|í)a|Mirasierra|Herrera ?Oria|Barrio ?del ?Pilar|Ventilla|Plaza ?de ?Castilla|Duque ?de ?Pastrana|P(i|í)o ?XII|Colombia|Concha ?Espina|Cruz ?del ?Rayo|Avenida ?de ?Am(e|é)rica|N(u|ú)ñez ?de ?Balboa|Pr(i|í)ncipe ?de ?Vergara|Ibiza|Sainz ?de ?Baranda|Estrella|Vinateros|Artilleros|Pavones|Valdebernardo|Vic(a|á)lvaro|San ?Cipriano|Puerta ?de ?Arganda|Rivas-Urbanizaciones|Rivas ?Futura|Rivas-Vaciamadrid|La ?Poveda|Arganda ?del ?Rey",
        "L10": "l(i|í)neas? ?(^|\D)10(\D|$)| L ?(^|\D)10(\D|$)|Hospital ?Infanta ?Sof(i|í)a|Reyes ?Cat(o|ó)licos|Baunatal|Manuel ?de ?Falla|Marqu(e|é)s ?de ?la ?Valdavia|La ?Moraleja|La ?Granja|Ronda ?de ?la ?Comunicaci(o|ó)n|Las ?Tablas|Montecarmelo|Tres ?Olivos|Fuencarral|Begoña|Chamart(i|í)n|Plaza ?de ?Castilla|Cuzco|Santiago ?Bernab(e|é)u|Nuevos ?Ministerios|Gregorio ?Marañ(o|ó)n|Alonso ?Mart(i|í)nez|Tribunal|Plaza ?de ?España|Pr(i|í)ncipe ?P(i|í)o|Lago|Bat(a|á)n|Casa ?de ?Campo|Colonia ?Jard(i|í)n|Aviaci(o|ó)n ?Española|Cuatro ?Vientos|Joaqu(i|í)n ?Vilumbrales|Puerta ?del ?Sur",
        "L11": "l(i|í)neas? ?(^|\D)11(\D|$)| L ?(^|\D)11(\D|$)|Plaza ?El(i|í)ptica|Abrantes|Pan ?Bendito|San ?Francisco|Carabanchel ?Alto|La ?Peseta|La ?Fortuna",
        "L12": "l(i|í)neas? ?(^|\D)12(\D|$)| L ?(^|\D)12(\D|$)|Alcorc(o|ó)n ?Central|Parque ?Lisboa|Parque ?Oeste|Puerta ?del ?Sur|San ?Nicasio|Universidad ?Rey ?Juan ?Carlos|Legan(e|é)s ?Central|M(o|ó)stoles ?Central|Hospital ?Severo ?Ochoa|Pradillo|Casa ?del ?Reloj|Hospital ?de ?M(o|ó)stoles|Juli(a|á)n ?Besteiro|Manuela ?Malasaña|El ?Carrascal|Loranca|El ?Bercial|Hospital ?de ?Fuenlabrada|Los ?Espartales|Parque ?Europa|El ?Casar|Fuenlabrada ?Central|Juan ?de ?la ?Cierva|Parque ?de ?los ?Estados|Getafe ?Central|Alonso ?de ?Mendoza|Arroyo ?Culebro|Conservatorio",
        "ML1": "metro ?ligero ?(^|\D)1(\D|$)| ml ?(^|\D)1(\D|$)|Pinar ?de ?Chamart(i|í)n|Fuente ?de ?la ?Mora|Virgen ?del ?Cortijo|Antonio ?Saura|Álvarez ?de ?Villaamil|Blasco ?Ibañez|Mar(i|í)a ?Tudor|Palas ?de ?Rey|Las ?Tablas",
        "ML2": "metro ?ligero ?(^|\D)2(\D|$)| ml ?(^|\D)2(\D|$)|Colonia ?Jard(i|í)n|Prado ?de ?la ?Vega|Colonia ?de ?los ?Ángeles|Prado ?del ?Rey|Somosaguas ?Sur|Somosaguas ?Centro|Pozuelo ?Oeste|Prado ?de ?las ?Bodegas|B(e|é)lgica|Dos ?Castillas|Campus ?de ?Somosaguas|Avenida ?de ?Europa|Berna|Estaci(o|ó)n ?de ?Aravaca",
        "ML3": "metro ?ligero ?(^|\D)3(\D|$)| ml ?(^|\D)3(\D|$)|Colonia ?Jard(i|í)n|Ciudad ?de ?la ?Imagen|Jos(e|é) ?Isbert|Ciudad ?del ?Cine|Cocheras|Retamares|Retamares ?Oeste|Montepr(i|í)ncipe|Ventorro ?del ?Cano|Prado ?del ?Espino|Ciudad ?Financiera ?Este|Cantabria|Ciudad ?Financiera ?Oeste|Ferial ?de ?Boadilla|Boadilla ?Centro|Nuevo ?Mundo|Siglo ?XXI|Infante ?Don ?Luis|Puerta ?de ?Boadilla"
    }
    tags = []
    for key, expr in regex_dict.items():
        if re.search(expr, text, re.IGNORECASE):
            tags.append(key)
    return tags

from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
generate_lines_udf = udf(generate_line_tags, ArrayType(StringType()))

In [66]:
from pyspark.sql.functions import explode
aux = mentions
            .withColumn('lines_array', generate_lines_udf(col('full_text')))
            .select('*', explode(col('lines_array')))

+----------------------------------+------------------------------+-------------------+---------------------+------------------+--------------------+----------------+-----------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+----------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [72]:
mentions.join(tweet_dim,
                mentions.id == tweet_dim.tweet_id,
              'inner'
             ) \
        .join(class_dim,
                class_dim.class_name == col('`classification.naive_bayes.result`'),
              'inner'
        )  \
        .withColumn('lines_array', generate_lines_udf(col('full_text'))) \
        .select(monotonically_increasing_id().alias('issue_id'), 'tweet_id', 'class_id', explode(col('lines_array')).alias('line')) \
        .join(lines_dim,
              col('line') == lines_dim.line_name
             ) \
        .show(10)

+-------------+-------------------+--------+----+---------+-------+
|     issue_id|           tweet_id|class_id|line|line_name|line_id|
+-------------+-------------------+--------+----+---------+-------+
|1219770712064|1062662964489129984|       1|  L1|       L1|     14|
|1219770712065|1062662964489129984|       1|  L2|       L2|     12|
|1219770712066|1062662964489129984|       1|  L3|       L3|     10|
|1219770712067|1062662964489129984|       1|  L9|       L9|      0|
|1219770712068|1064889115932528641|       1| L10|      L10|      9|
|1219770712069|1065522105737990144|       1|  L9|       L9|      0|
|1219770712070|1068495447407366145|       1|  L3|       L3|     10|
|1219770712071|1068495447407366145|       1|  L6|       L6|      8|
|1219770712072|1068864540367036416|       1|  L1|       L1|     14|
|1219770712073|1068894720062095361|       1|  L1|       L1|     14|
+-------------+-------------------+--------+----+---------+-------+
only showing top 10 rows



### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.